In [1]:
from PIL import ImageFont, ImageDraw, Image
from fontTools.ttLib import TTFont

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import time
import datetime
import multiprocessing

In [2]:
VOCAB = 28; EBD_DIM = 256; UNIT_DIM = 128; BATCH_SIZE = 256

## Load Data

In [3]:
class Glyph(object):
    # transform character to bitmap
    def __init__(self, fonts, size=64):
        # load fonts, size. We will use 2 fonts for all CJK characters, so keep 2 codepoint books.
        self.codepoints = [set() for _ in fonts]
        self.size = int(size * 0.8)
        self.size_img = size
        self.pad = (size - self.size) // 2
        self.fonts = [ImageFont.truetype(f, self.size) for f in fonts]
        # use a cache to reduce computation if duplicated characters encountered.
        self.cache = {}
        for cp, font in zip(self.codepoints, fonts):
            font = TTFont(font)
            # store codepoints in font cmap into self.codepoints
            for cmap in font['cmap'].tables:
                if not cmap.isUnicode():
                    continue
                for k in cmap.cmap:
                    cp.add(k)
    
    def draw(self, ch):
        if ch in self.cache:
            return self.cache[ch]
        # search among fonts, use the first found
        exist = False
        for i in range(len(self.codepoints)):
            if ord(ch) in self.codepoints[i]:
                font = self.fonts[i]
                exist = True
                break
        if not exist:
            return None

        img = Image.new('L', (self.size_img, self.size_img), 0)
        draw = ImageDraw.Draw(img)
        (width, baseline), (offset_x, offset_y) = font.font.getsize(ch)
        draw.text((self.pad - offset_x, self.pad - offset_y + 4), ch, font=font, fill=255, stroke_fill=255) 
        img_array = np.array(img.getdata(), dtype='float32').reshape((self.size_img, self.size_img)) / 255
        self.cache[ch] = img_array

        return img_array

In [4]:
glyphbook = Glyph(['data/fonts/HanaMinA.otf', 'data/fonts/HanaMinB.otf'])

def _mapping(item):
    char, code, dup_total, dup_curr = item
    glyph = glyphbook.draw(char)
    if glyph is not None:
        return glyph, code, dup_total, dup_curr

def preprocess_chart(chart, cores=multiprocessing.cpu_count()):
    glyphs = []; codes = []
    dup_total = []; dup_curr = []
    with multiprocessing.Pool(processes=cores) as pool:
        for item in pool.map(_mapping, chart.values):
            if item is not None:
                glyphs.append(item[0])
                codes.append(item[1])
                dup_total.append(item[2])
                dup_curr.append(item[3])
    return np.expand_dims(np.array(glyphs), -1), np.array(codes), np.array(dup_total), np.array(dup_curr)

In [5]:
def tokenizer(code_table):
    # Cangjie code consists only of a-z, with maximum length of 5, minimum of 1
    # start with 0, a-z are 1-26, end and padding are 27
    tokens = np.expand_dims(np.zeros(code_table.shape, dtype='int64'), -1)
    code_index = list(map(lambda x: list(map(lambda y: ord(y) - 96, list(x))) + [27] * (5-len(x)), code_table))
    tokens = np.append(tokens, np.array(code_index), axis=-1)
    return tokens

In [6]:
code_chart = pd.read_csv('data/cangjie6.txt', delimiter='\t', header=None, names=['Char', 'Code'], keep_default_na=False)

In [7]:
count = {}
for char, code in code_chart.values:
    if char in count:
        count[char].append(code)
        count[char].sort(key=len)
        count[char].sort(key=lambda x: (len(x), x))
    else:
        count[char] = [code]

In [8]:
MAX_LEN = code_chart.Code.map(len).max()
MAX_DUP = max(map(lambda x: len(x), count.values()))

In [9]:
code_chart['DuplicateTotal'] = code_chart['Char'].map(count).map(len).copy()
code_chart['DuplicateCurrent'] = code_chart.apply(lambda x: count[x['Char']].index(x['Code']) + 1, axis=1).copy()

In [10]:
glyphs, codes, dups_total, dups_curr = preprocess_chart(code_chart)
tokens = tokenizer(codes)
lengths = np.array([len(list(filter(lambda i: i < VOCAB - 1 and i > 0, x))) for x in tokens])
lengths = np.array([np.identity(MAX_LEN)[i-1] for i in lengths], dtype='int64')
dups_total = np.array([np.identity(MAX_DUP)[i-1] for i in dups_total], dtype='int64')
dups_curr = np.array([np.identity(MAX_DUP)[i-1] for i in dups_curr], dtype='int64')
del code_chart, codes, count

In [11]:
(train_glyphs, validation_glyphs,
 train_tokens, validation_tokens,
 train_lengths, validation_lengths,
 train_dups_total, validation_dups_total,
 train_dups_curr, validation_dups_curr) = train_test_split(
    glyphs, tokens, lengths, dups_total, dups_curr, test_size=0.1, random_state=1225)
del glyphs, tokens, lengths, dups_total, dups_curr

In [12]:
num_steps = train_glyphs.shape[0] // BATCH_SIZE + min(1, train_glyphs.shape[0] % BATCH_SIZE)
num_steps_val = validation_glyphs.shape[0] // BATCH_SIZE + min(1, validation_glyphs.shape[0] % BATCH_SIZE)

dataset = tf.data.Dataset.from_tensor_slices((train_glyphs, train_tokens, train_lengths, train_dups_total, train_dups_curr))
dataset = dataset.shuffle(train_glyphs.shape[0]).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((validation_glyphs, validation_tokens, validation_lengths, validation_dups_total, validation_dups_curr))
val_dataset = val_dataset.shuffle(validation_glyphs.shape[0]).batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

del train_glyphs, validation_glyphs, train_tokens, validation_tokens, train_lengths, validation_lengths
del train_dups_total, validation_dups_total, train_dups_curr, validation_dups_curr

## Model

In [13]:
class Res_CNN(tf.keras.Model):
    def __init__(self, feature_dim, kernel_size):
        super(Res_CNN, self).__init__()
        self.cnn1 = tf.keras.layers.Convolution2D(feature_dim, kernel_size, padding='same')
        self.cnn2 = tf.keras.layers.Convolution2D(feature_dim, kernel_size, padding='same')
        self.cnn3 = tf.keras.layers.Convolution2D(feature_dim, kernel_size, padding='same')
        self.norm = tf.keras.layers.BatchNormalization()
        
    def call(self, x):
        x = self.cnn1(x)
        x_identity = tf.identity(x)
        x = self.cnn2(x)
        x_identity2 = tf.identity(x)
        x = self.cnn3(x + x_identity)
        x = self.norm(x + x_identity2)
        x = tf.nn.relu(x)
        return x

In [14]:
class CNN_Encoder(tf.keras.Model):
    # This is essentially a CNN layer, 
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.res_cnn1 = Res_CNN(embedding_dim // 16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        self.res_cnn2 = Res_CNN(embedding_dim // 4, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        self.res_cnn3 = Res_CNN(embedding_dim, (3, 3))
        self.fc = tf.keras.layers.Dense(embedding_dim, activation='relu')

    def call(self, x, training=True):
        # x shape after cnn1 == (batch_size, 64, 64, embedding_dim // 16)
        x = self.res_cnn1(x)
        # x shape after pool1 == (batch_size, 32, 32, embedding_dim // 16)
        x = self.pool1(x)
        
        # x shape after cnn2 == (batch_size, 32, 32, embedding_dim // 4)
        x = self.res_cnn2(x)
        # x shape after pool2 == (batch_size, 16, 16, embedding_dim // 4)
        x = self.pool2(x)
        
        # x shape after cnn3 == (batch_size, 16, 16, embedding_dim)
        x = self.res_cnn3(x)
        # reshape from (batch_size, 16, 16, embedding_dim) to (batch_size, 256, embedding_dim)
        x = tf.reshape(x, [x.shape[0], -1, x.shape[-1]])
        # x shape after fc == (batch_size, 256, embedding_dim)
        if training:
            x = tf.nn.dropout(x, rate=0.4)
        x = self.fc(x)
        return x

In [15]:
class Bahdanau_Attention(tf.keras.Model):
    def __init__(self, attention_dim):
        super(Bahdanau_Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(attention_dim)
        self.W2 = tf.keras.layers.Dense(attention_dim)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_Encoder output) shape == (batch_size, 256, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, 256, attention_dim)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        # attention_weights shape == (batch_size, 256, 1)
        # you get 1 at the last axis because you are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # context_vector shape after sum == (batch_size, embedding_dim)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [16]:
class Simple_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, max_length, hidden_size, vocab_size):
        super(Simple_Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.attention = Bahdanau_Attention(hidden_size)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu')
        self.fc2 = tf.keras.layers.Dense(vocab_size)
        
    def call(self, feature, position):
        # y shape (batch_size, hidden_size)
        y = self.embedding(position)
        # x shape (batch_size, embedding_dim)
        x, w = self.attention(feature, y)
        # x shape (batch_size, hidden_size)
        x = self.fc1(x)
        # x shape (batch_size, vocab_size)
        x = self.fc2(x)
        return x, w

In [17]:
class Length_Decoder(tf.keras.Model):
    def __init__(self, max_length):
        super(Length_Decoder, self).__init__()
        self.pool = tf.keras.layers.MaxPool2D((2, 2))
        self.fc1 = tf.keras.layers.Dense(max_length * 16, activation='relu')
        self.fc2 = tf.keras.layers.Dense(max_length * 16, activation='relu')
        self.fc3 = tf.keras.layers.Dense(max_length * 4, activation='relu')
        self.fc4 = tf.keras.layers.Dense(max_length)
        
    def call(self, x, d_t=None, d_c=None):
        x = tf.reshape(x, (x.shape[0], 16, 16, x.shape[-1]))
        x = self.pool(x) # shape = (batch_size, 8, 8, embedding_dim)
        x = self.fc1(x)
        x = tf.reshape(x, (x.shape[0], -1))
        if d_t != None and d_c != None:
            d = tf.concat([tf.cast(d_t, 'float32'), tf.cast(d_c, 'float32')], axis=-1)
            x = tf.concat([d, x], axis=-1)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        # shape = (batch_size, max_length)
        return x

In [18]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, hidden_size, vocab_size, max_length):
        super(RNN_Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(self.hidden_size, return_sequences=True,
                                        return_state=True, recurrent_initializer='glorot_uniform', dropout=0.3)
        self.gru2 = tf.keras.layers.GRU(self.hidden_size, return_sequences=True,
                                        return_state=True, recurrent_initializer='glorot_uniform')
        self.gru3 = tf.keras.layers.GRU(self.hidden_size, return_sequences=True,
                                        return_state=True, recurrent_initializer='glorot_uniform', dropout=0.3)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu')
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = Bahdanau_Attention(hidden_size)

    def call(self, x, l, d_t, d_c, features, hidden, training=True, teacher_forcing=True):
        # x is forward direction, y is beckward direction
        # defining attention as a separate model
        l = tf.cast(l, 'float32')
        hidden_0_with_length = tf.concat([l, hidden[0]], axis=-1)
        context_vector, attention_weights = self.attention(features, hidden_0_with_length)
        l = tf.expand_dims(l, 1)
        d = tf.expand_dims(tf.concat([tf.cast(d_t, 'float32'), tf.cast(d_c, 'float32')], axis=-1), 1)

        # x shape before is (batch_size, 1) since it is passed through one by one at a time
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        if teacher_forcing:
            x = self.embedding(x)
        else:
            x = tf.tensordot(x, self.embedding.weights[0], axes=[-1,0])
        # context_vector shape is (batch_size, embedding_dim)
        # x shape after concatenation == (batch_size, 1, embedding_dim + embedding_dim)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        # x shape is (batch_size, 1, hidden_size)
        # state is new hidden used in next step
        x, state1 = self.gru1(x, initial_state = hidden[0], training=training)
        x_identity = tf.identity(x)
        x = tf.concat([d, l, x], axis=-1)
        x, state2 = self.gru2(x, initial_state = hidden[1], training=training)
        x_identity2 = tf.identity(x)
        x, state3 = self.gru3(x + x_identity, initial_state = hidden[2], training=training)
        # x shape (batch_size, 1, max_length + hidden_size)
        x = tf.concat([d, l, x + x_identity2], axis=-1)
        x = tf.reshape(x, (x.shape[0], -1))
        # x shape (batch_size, hidden_size)
        x = self.fc1(x)
        # x shape (batch_size, vocab_size)
        x = self.fc2(x)

        return x, [state1, state2, state3], attention_weights

    def reset_state(self, batch_size):
        # generate new hidden layer with different batch size
        return [tf.zeros((batch_size, self.hidden_size)) for _ in range(3)]

## Graph

In [19]:
optimizer_step1 = tf.keras.optimizers.Adam()
optimizer_step2 = tf.keras.optimizers.Adam()
optimizer_length = tf.keras.optimizers.Adam()
optimizer_dups = tf.keras.optimizers.Adam()

### Step 1

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    loss_ = loss_object(real, pred)
    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    pred_index = tf.math.argmax(pred, axis=-1)
    return tf.math.reduce_mean(tf.cast(pred_index == real, tf.float32))

In [21]:
@tf.function
def train_step1(glyph, target, length, total_dups, curr_dups):
    loss = 0; accuracy = 0
    with tf.GradientTape() as tape:
        features = encoder(glyph)
        for i in range(1, target.shape[1]):
            position = tf.convert_to_tensor(np.repeat(i-1, target.shape[0]), dtype='int64')
            prediction, weight = simple_decoder(features, position)
            loss += loss_function(target[:, i], prediction)
            accuracy += accuracy_function(target[:, i], prediction)

    trainable_variables = simple_decoder.trainable_variables + encoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer_step1.apply_gradients(zip(gradients, trainable_variables))
    
    with tf.GradientTape() as tape_length:
        length_pred = length_decoder(features, total_dups, curr_dups)
        loss_length = loss_function(tf.math.argmax(length, axis=-1), length_pred)

    gradients_length = tape_length.gradient(loss_length, length_decoder.trainable_variables)
    optimizer_length.apply_gradients(zip(gradients_length, length_decoder.trainable_variables))
    
    with tf.GradientTape() as tape_dups:
        dups_pred = dup_decoder(features)
        loss_dups = loss_function(tf.math.argmax(total_dups, axis=-1), dups_pred)

    gradients_dups = tape_dups.gradient(loss_dups, dup_decoder.trainable_variables)
    optimizer_dups.apply_gradients(zip(gradients_dups, dup_decoder.trainable_variables))
    
    return loss / (target.shape[1] - 1), accuracy / (target.shape[1] - 1)

In [22]:
@tf.function
def validation_step1(glyph, target):
    loss = 0; accuracy = 0
    feature = encoder(glyph, training=False)
    for i in range(1, target.shape[1]):
        position = tf.convert_to_tensor(np.repeat(i-1, target.shape[0]), dtype='int64')
        prediction, weight = simple_decoder(feature, position)
        loss += loss_function(target[:, i], prediction)
        accuracy += accuracy_function(target[:, i], prediction)
    return loss / (target.shape[1] - 1), accuracy / (target.shape[1] - 1)

In [23]:
def step1(epoch):
    start = time.time()
    total_loss = 0; val_loss = 0
    total_accuracy = 0; val_accuracy = 0

    for (batch, (glyph_tensor, target, length, total_dups, curr_dups)) in enumerate(dataset):
        t_loss, accuracy = train_step1(glyph_tensor, target, length, total_dups, curr_dups)
        total_loss += t_loss
        total_accuracy += accuracy
        print('Epoch {}, Train Loss {:.4f}, Accuracy {:.2%}; progress {:.1%}, taken {:.0f} sec'.format(
            epoch + 1, total_loss/batch, total_accuracy / batch, batch / num_steps, time.time() - start), end='\r')
    
    for (glyph_tensor, target, _, _, _) in val_dataset:
        t_loss, accuracy = validation_step1(glyph_tensor, target)
        val_loss += t_loss
        val_accuracy += accuracy
   
    # storing the epoch end loss value to plot later 
    ckpt_manager_step1.save()

    print ('Epoch {}, Train Loss {:.4f}, Accuracy {:.2%} | Validation Loss {:.4f}, Accuracy {:.2%}; taken {:.0f} sec'.format(
        epoch+1, total_loss/num_steps, total_accuracy/num_steps, val_loss/num_steps_val, val_accuracy/num_steps_val, time.time() - start))

### Step 2

In [24]:
def predict(features, max_length, length, total_dups, curr_dups):
    # start with 0
    dec_input = tf.constant([[[1] + [0] * (VOCAB - 1)]] * features.shape[0], dtype='float32')
    hidden = decoder.reset_state(batch_size=features.shape[0])
    probability = tf.convert_to_tensor([1]*features.shape[0], dtype='float32')
    # iterate predictions, no teacher forcing here
    for i in range(max_length):
        prediction, hidden, attention_weights = decoder(
            tf.expand_dims(dec_input[:, i, :], 1), length, total_dups, curr_dups, features, hidden, training=False, teacher_forcing=False)
        # we need deterministic result
        prediction = tf.math.softmax(prediction, axis=-1)
        probability *= tf.math.reduce_max(prediction, axis=-1)
        dec_input = tf.concat([dec_input, tf.expand_dims(prediction, 1)], axis=1)
    return tf.math.argmax(dec_input, axis=-1), probability

In [25]:
def predict_next(features, target, length, total_dups, curr_dups, training=True, teacher_forcing=True):
    hidden = decoder.reset_state(batch_size=features.shape[0])
    predictions = tf.constant(0, dtype='float32', shape=(features.shape[0], 1, VOCAB))
    if not teacher_forcing:
        previous = tf.constant([[[1] + [0] * (VOCAB - 1)]] * features.shape[0], dtype='float32')
    for i in range(target.shape[1]-1):
        if teacher_forcing:
            previous = tf.expand_dims(target[:, i], 1)
        prediction, hidden, attention_weights = decoder(
            previous, length, total_dups, curr_dups, features, hidden, training=training, teacher_forcing=teacher_forcing)
        predictions = tf.concat([predictions, tf.expand_dims(prediction, 1)], axis=1)
        if not teacher_forcing:
            previous = tf.expand_dims(tf.math.softmax(prediction, axis=-1), 1)
    return predictions[:, 1:, :]

In [26]:
def loss_function_step2(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    loss_ = tf.reduce_mean(loss_, axis=0)
    return tf.reduce_sum(loss_)

def accuracy_function_step2(real, pred):
    accuracy = tf.math.reduce_all(pred == real, 1)
    return tf.math.reduce_mean(tf.cast(accuracy, tf.float32))

In [27]:
@tf.function
def train_step2(glyph_tensor, target, length, total_dups, curr_dups, teacher_forcing=True):

    with tf.GradientTape() as tape:
        features = encoder(glyph_tensor)
        
        with tf.GradientTape() as tape_dups:
            dups_pred = dup_decoder(features)
            loss_dups = loss_function(tf.math.argmax(total_dups, axis=-1), dups_pred)
        dups_pred = tf.nn.softmax(dups_pred, axis=-1)
        
        with tf.GradientTape() as tape_length:
            if teacher_forcing:
                length_pred = length_decoder(features, d_t=total_dups, d_c=curr_dups)
            else:
                length_pred = length_decoder(features, d_t=dups_pred, d_c=curr_dups)
            loss_length = loss_function(tf.math.argmax(length, axis=-1), length_pred)
        length_pred = tf.nn.softmax(length_pred, axis=-1)
        
        if teacher_forcing:
            predictions = predict_next(features, target, length, total_dups, curr_dups)
        else:
            predictions = predict_next(features, target, length_pred, dups_pred, curr_dups, teacher_forcing=teacher_forcing)
        loss = loss_function_step2(target[:, 1:], predictions)
    
    
    trainable_variables = decoder.trainable_variables + encoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer_step2.apply_gradients(zip(gradients, trainable_variables))

    gradients_length = tape_length.gradient(loss_length, length_decoder.trainable_variables)
    optimizer_length.apply_gradients(zip(gradients_length, length_decoder.trainable_variables))
    
    gradients_dups = tape_dups.gradient(loss_dups, dup_decoder.trainable_variables)
    optimizer_dups.apply_gradients(zip(gradients_dups, dup_decoder.trainable_variables))
    
    # calculate accuracy based on the code's whole string
    predictions_id, _ = predict(features, MAX_LEN, length_pred, dups_pred, curr_dups)
    accuracy = accuracy_function_step2(predictions_id, target)
    accuracy_length = accuracy_function(tf.math.argmax(length, axis=-1), length_pred)
    accuracy_dups = accuracy_function(tf.math.argmax(total_dups, axis=-1), dups_pred)

    return loss / (target.shape[1] - 1), accuracy, accuracy_length, accuracy_dups

In [28]:
@tf.function
def validation_step2(glyph_tensor, target, length, total_dups, curr_dups):
    features = encoder(glyph_tensor, training=False)
    dups_pred = tf.nn.softmax(dup_decoder(features), axis=-1)
    length_pred = tf.nn.softmax(length_decoder(features, d_t=dups_pred, d_c=curr_dups), axis=-1)
    predictions = predict_next(features, target, length_pred, dups_pred, curr_dups, training=False, teacher_forcing=False)
    loss = loss_function_step2(target[:, 1:], predictions)

    # calculate accuracy based on the code's whole string
    predictions_id, _ = predict(features, MAX_LEN, length_pred, dups_pred, curr_dups)
    accuracy = accuracy_function_step2(predictions_id, target)
    accuracy_length = accuracy_function(tf.math.argmax(length, axis=-1), length_pred)
    accuracy_dups = accuracy_function(tf.math.argmax(total_dups, axis=-1), dups_pred)
    
    return loss / (target.shape[1] - 1), accuracy, accuracy_length, accuracy_dups

In [29]:
def step2(epoch, teacher_forcing=True):
    start = time.time()
    total_loss = 0; val_loss = 0; dups_accu = 0; val_dups_accu = 0
    total_accuracy = 0; val_accuracy = 0; len_accu = 0; val_len_accu = 0

    for (batch, (glyph_tensor, target, length, total_dups, curr_dups)) in enumerate(dataset):
#        if batch == 0:
#            tf.summary.trace_on(graph=True, profiler=True)
        t_loss, accuracy, accuracy_length, accuracy_dups = train_step2(glyph_tensor, target, length, total_dups, curr_dups, teacher_forcing=teacher_forcing)
#        if batch == 0:
#            with graph_summary_writer.as_default():
#                tf.summary.trace_export(name="train_trace", step=epoch, profiler_outdir=graph_log_dir)
#            tf.summary.trace_off()
        total_loss += t_loss; total_accuracy += accuracy
        len_accu += accuracy_length; dups_accu += accuracy_dups
        
        print('Epoch {}, Train Loss {:.4f}, Accuracy {:.2%}; Length Accuracy {:.2%}, Dups Accuracy {:.2%}; progress {:.1%}, taken {:.0f} sec'.format(
            epoch + 1, total_loss/batch, total_accuracy/batch, len_accu/batch, dups_accu/batch, batch/num_steps, time.time() - start), end='\r')
    
    for (glyph_tensor, target, length, total_dups, curr_dups) in val_dataset:
        t_loss, accuracy, accuracy_length, accuracy_dups = validation_step2(glyph_tensor, target, length, total_dups, curr_dups)
        val_loss += t_loss; val_accuracy += accuracy
        val_len_accu += accuracy_length; val_dups_accu += accuracy_dups
   
    # storing the epoch end loss value to plot later
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', (total_loss / num_steps), step=epoch)
        tf.summary.scalar('accuracy', (total_accuracy / num_steps), step=epoch)
        tf.summary.scalar('length_accuracy', (len_accu / num_steps), step=epoch)
        tf.summary.scalar('duplication_accuracy', (dups_accu / num_steps), step=epoch)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', val_loss / num_steps_val, step=epoch)
        tf.summary.scalar('accuracy', val_accuracy / num_steps_val, step=epoch)
        tf.summary.scalar('length_accuracy', val_len_accu / num_steps_val, step=epoch)
        tf.summary.scalar('duplication_accuracy', val_dups_accu / num_steps_val, step=epoch)
    
    ckpt_manager_step2.save()

    print('Epoch {}, Train Loss {:.4f}, Accuracy {:.2%}; Length Accuracy {:.2%}, Dups Accuracy {:.2%} | Validation Loss {:.4f}, Accuracy {:.2%}; Length Accuracy {:.2%}, Dups Accuracy {:.2%}; taken {:.0f} sec'.format(
        epoch + 1, total_loss/num_steps, total_accuracy/num_steps, len_accu/num_steps, dups_accu/num_steps, 
        val_loss/num_steps_val, val_accuracy/num_steps_val, val_len_accu/num_steps_val, val_dups_accu/num_steps_val, time.time() - start))

## Training

In [30]:
encoder = CNN_Encoder(embedding_dim = EBD_DIM)
simple_decoder = Simple_Decoder(embedding_dim = EBD_DIM, max_length = MAX_LEN, hidden_size = UNIT_DIM, vocab_size = VOCAB)
length_decoder = Length_Decoder(max_length = MAX_LEN)
dup_decoder = Length_Decoder(max_length = MAX_DUP)
decoder = RNN_Decoder(embedding_dim=EBD_DIM, hidden_size=UNIT_DIM, max_length = MAX_LEN, vocab_size=VOCAB)

In [31]:
# use a checkpoint to store weights
checkpoint_path_step1 = './checkpoints/train_step1'
ckpt_step1 = tf.train.Checkpoint(encoder=encoder, decoder=simple_decoder, length_decoder=length_decoder,
    dup_decoder=dup_decoder, optimizer=optimizer_step1, optimizer_length=optimizer_length, optimizer_dups=optimizer_dups)
ckpt_manager_step1 = tf.train.CheckpointManager(ckpt_step1, checkpoint_path_step1, max_to_keep=5)

In [32]:
# use a checkpoint to store weights
checkpoint_path_step2 = "./checkpoints/train_step2"
ckpt_step2 = tf.train.Checkpoint(encoder=encoder, decoder=decoder, length_decoder=length_decoder,
    dup_decoder=dup_decoder, optimizer=optimizer_step2, optimizer_length=optimizer_length, optimizer_dups=optimizer_dups)
ckpt_manager_step2 = tf.train.CheckpointManager(ckpt_step2, checkpoint_path_step2, max_to_keep=5)

In [33]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
#graph_log_dir = 'logs/gradient_tape/' + current_time + '/func'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
#graph_summary_writer = tf.summary.create_file_writer(graph_log_dir)

### Step 1

In [34]:
EPOCHS_STEP1 = 20

epoch_step1 = 0
if ckpt_manager_step1.latest_checkpoint:
    epoch_step1 = int(ckpt_manager_step1.latest_checkpoint.split('-')[-1])
    ckpt_step1.restore(ckpt_manager_step1.latest_checkpoint)

In [35]:
while epoch_step1 < EPOCHS_STEP1:
    step1(epoch_step1)
    epoch_step1 += 1

Epoch 1, Train Loss 2.8708, Accuracy 20.49% | Validation Loss 2.5070, Accuracy 31.04%; taken 92 sec
Epoch 2, Train Loss 1.7768, Accuracy 48.47% | Validation Loss 1.3149, Accuracy 64.21%; taken 79 sec
Epoch 3, Train Loss 1.1622, Accuracy 65.67% | Validation Loss 0.9238, Accuracy 75.28%; taken 78 sec
Epoch 4, Train Loss 0.9092, Accuracy 72.87% | Validation Loss 0.7718, Accuracy 79.65%; taken 78 sec
Epoch 5, Train Loss 0.7747, Accuracy 76.71% | Validation Loss 0.6806, Accuracy 82.36%; taken 78 sec
Epoch 6, Train Loss 0.6876, Accuracy 79.26% | Validation Loss 0.6158, Accuracy 84.05%; taken 78 sec
Epoch 7, Train Loss 0.6279, Accuracy 80.96% | Validation Loss 0.5577, Accuracy 86.00%; taken 79 sec
Epoch 8, Train Loss 0.5785, Accuracy 82.51% | Validation Loss 0.5404, Accuracy 86.33%; taken 79 sec
Epoch 9, Train Loss 0.5439, Accuracy 83.42% | Validation Loss 0.4994, Accuracy 87.60%; taken 79 sec
Epoch 12, Train Loss 0.4708, Accuracy 85.54% | Validation Loss 0.4768, Accuracy 87.96%; taken 78 sec

### Step 2

In [36]:
EPOCHS_STEP2 = 120

epoch_step2 = 0
if ckpt_manager_step2.latest_checkpoint:
    epoch_step2 = int(ckpt_manager_step2.latest_checkpoint.split('-')[-1])
    ckpt_step2.restore(ckpt_manager_step2.latest_checkpoint)

In [37]:
while epoch_step2 < EPOCHS_STEP2:
    if epoch_step2 < EPOCHS_STEP2 // 2:
        step2(epoch_step2)
    else:
        step2(epoch_step2, teacher_forcing=False)
    epoch_step2 += 1

Epoch 1, Train Loss 1.4044, Accuracy 21.30%; Length Accuracy 88.10%, Dups Accuracy 91.19% | Validation Loss 0.6228, Accuracy 52.03%; Length Accuracy 93.00%, Dups Accuracy 94.96%; taken 126 sec
Epoch 2, Train Loss 0.4880, Accuracy 54.24%; Length Accuracy 91.89%, Dups Accuracy 93.88% | Validation Loss 0.5192, Accuracy 62.06%; Length Accuracy 92.94%, Dups Accuracy 95.66%; taken 89 sec
Epoch 3, Train Loss 0.3897, Accuracy 61.67%; Length Accuracy 92.51%, Dups Accuracy 94.41% | Validation Loss 0.5496, Accuracy 65.29%; Length Accuracy 92.32%, Dups Accuracy 95.84%; taken 89 sec
Epoch 4, Train Loss 0.3390, Accuracy 65.58%; Length Accuracy 92.56%, Dups Accuracy 94.61% | Validation Loss 0.4519, Accuracy 69.69%; Length Accuracy 94.62%, Dups Accuracy 95.72%; taken 89 sec
Epoch 5, Train Loss 0.3046, Accuracy 68.33%; Length Accuracy 92.81%, Dups Accuracy 94.86% | Validation Loss 0.4348, Accuracy 72.54%; Length Accuracy 94.57%, Dups Accuracy 96.17%; taken 89 sec
Epoch 6, Train Loss 0.2807, Accuracy 70

## Testing

### Whether prediction probability a good indicator of correctness?

In [38]:
@tf.function
def correctness_probs(glyph_tensor, target, length, total_dups, curr_dups):
    features = encoder(glyph_tensor, training=False)
    dups_pred = tf.nn.softmax(dup_decoder(features), axis=-1)
    length_pred = tf.nn.softmax(length_decoder(features, d_t=dups_pred, d_c=curr_dups), axis=-1)

    # calculate accuracy based on the code's whole string
    predictions_id, probabilities = predict(features, MAX_LEN, length_pred, dups_pred, curr_dups)
    correctness = tf.math.reduce_all(predictions_id == target, 1)
    true_probs = tf.boolean_mask(probabilities, correctness)
    false_probs = tf.boolean_mask(probabilities, tf.math.logical_not(correctness))
    
    return true_probs, false_probs

In [39]:
correct_probs = np.array([])
wrong_probs = np.array([])
for (glyph_tensor, target, length, total_dups, curr_dups) in val_dataset:
    t_p, f_p = correctness_probs(glyph_tensor, target, length, total_dups, curr_dups)
    correct_probs = np.append(correct_probs, t_p.numpy())
    wrong_probs = np.append(wrong_probs, f_p.numpy())

In [40]:
for i in range(1, 10):
    print('{}th percentile: correct predictions {:.2%}, wrong predictions {:.2%}'.format(
        10*i, np.percentile(correct_probs, 10*i), np.percentile(wrong_probs, 10*i)))

10th percentile: correct predictions 84.82%, wrong predictions 28.65%
20th percentile: correct predictions 95.11%, wrong predictions 39.88%
30th percentile: correct predictions 97.69%, wrong predictions 48.79%
40th percentile: correct predictions 98.71%, wrong predictions 55.86%
50th percentile: correct predictions 99.20%, wrong predictions 64.77%
60th percentile: correct predictions 99.49%, wrong predictions 73.52%
70th percentile: correct predictions 99.69%, wrong predictions 81.87%
80th percentile: correct predictions 99.82%, wrong predictions 90.05%
90th percentile: correct predictions 99.90%, wrong predictions 97.07%


### Have a look at the actual use

In [41]:
@tf.function
def test(glyph):
    features = encoder(glyph, training=False)
    total_dups = tf.nn.softmax(dup_decoder(features), axis=-1)
    dups_dict = tf.math.argmax(total_dups, axis=-1)
    max_dup = tf.math.reduce_max(dups_dict) + 1
    
    results = tf.zeros((glyph.shape[0], max_dup, MAX_LEN + 1), dtype='int64')
    probs = tf.zeros((glyph.shape[0], max_dup), dtype='float32')
    identity_matrix = tf.convert_to_tensor(np.identity(MAX_DUP), dtype='int64')
    
    for i in range(max_dup):
        curr_dups = tf.math.minimum(tf.math.argmax(total_dups, axis=-1), i)
        curr_dups = tf.nn.embedding_lookup(identity_matrix, curr_dups)
        length = tf.nn.softmax(length_decoder(features, total_dups, curr_dups), axis=-1)
        test_result, prob = predict(features, MAX_LEN, length, total_dups, curr_dups)
        results = tf.concat([results[:, :i, :], tf.expand_dims(test_result, axis=1), tf.zeros((glyph.shape[0], max_dup - i - 1, MAX_LEN + 1), dtype='int64')], axis=1)
        probs = tf.concat([probs[:, :i], tf.expand_dims(prob, axis=1), tf.zeros((glyph.shape[0], max_dup - i - 1), dtype='float32')], axis=1)
    return results, probs, dups_dict

In [42]:
def evaluate(word):
    test_input = []
    for char in word:
        glyph = glyphbook.draw(char)
        if glyph is not None:
            test_input.append(glyph)
        else:
            raise ValueError('Character {} unsupported.'.format(char))
    test_input = np.expand_dims(test_input, -1)
    
    def decode(indexes):
        code = ''
        for i in indexes:
            if i <= 0:
                continue
            elif i >= 27:
                break
            else:
                code += chr(i + 96)
        return code
    
    results, probs, dups_dict = test(test_input)
    results = results.numpy()
    probs = probs.numpy()
    dups_dict = dups_dict.numpy()
    
    final_result = []
    for i in range(results.shape[0]):
        final_result.append([])
        for j in range(results.shape[1]):
            if j <= dups_dict[i]:
                final_result[-1].append([decode(results[i, j, :]), probs[i, j]])
    
    return final_result

In [ ]:
evaluate('仁義禮智信温良恭儉讓')

[[['omm', 0.9523368]],
 [['tghi', 0.8739416]],
 [['iftwt', 0.99626595], ['mftwt', 0.99561006]],
 [['ora', 0.98931175]],
 [['oymr', 0.99481773]],
 [['eabt', 0.99797684]],
 [['iav', 0.9749448]],
 [['tcp', 0.99844986]],
 [['ooro', 0.9937146]],
 [['yryrv', 0.9985458]]]